In [ ]:
import pandas as pd

df0 = pd.read_csv('10Targcleaned.csv') #Target
df1 = pd.read_csv('1000014cleaned.csv') #Parkhäuser
df2 = pd.read_csv('1000048cleaned.csv') #
df3 = pd.read_csv('1000050cleaned.csv') #
df4 = pd.read_csv('1000051cleaned.csv') #
df5 = pd.read_csv('10Flugbewegungencleaned.csv', delimiter=';') #Flugbewegungen(ändert sich wahrscheinlich noch)
df6 = pd.read_csv('Gasverbrauch_cleaned.csv') #Gasverbrauch
df7 = pd.read_csv('merged_df.csv') #Verkehr
df8 = pd.read_csv('Schallmessungen_cleaned.csv') #Schallmessungen
df9 = pd.read_csv('Stromverbrauch_Basel_hourly_cleaned.csv') #Stromverbrauch
df11 = pd.read_csv('Wetterdaten_Basel_cleanded22_24.csv') #Wetterdaten

In [ ]:
# Check the columns of df0
print(df0.columns)

# If the column name is different, use the correct column name
# Assuming the correct column name is 'Date/Time'
df0['Datum'] = pd.to_datetime(df0['Datum']) + pd.DateOffset(hours=-1)
df0

Index(['Datum', 'PM10 (Stundenmittelwerte)', 'PM2.5 (Stundenmittelwerte)'], dtype='object')


,Datum,PM10 (Stundenmittelwerte),PM2.5 (Stundenmittelwerte)
0,2021-12-31 23:00:00,83.840,71.870
1,2022-01-01 00:00:00,91.973,78.909
2,2022-01-01 01:00:00,68.396,58.632
3,2022-01-01 02:00:00,49.625,42.563
4,2022-01-01 03:00:00,41.854,35.799
...,...,...,...
25354,2024-11-22 09:00:00,9.305,7.167
25355,2024-11-22 10:00:00,10.314,7.574
25356,2024-11-22 11:00:00,7.101,5.376
25357,2024-11-22 12:00:00,5.904,5.010


In [3]:
# List of dataframes
dfs = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df11]

# Loop through each dataframe
for i, df in enumerate(dfs):
    first_col = df.columns[0]
    df[first_col] = pd.to_datetime(df[first_col], errors='coerce')
    df.rename(columns={first_col: 'DaTii'}, inplace=True)
    dfs[i] = df

# Assign the transformed dataframes back to their original names
df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df11 = dfs

#. Ensure all 'DaTii' columns are localized to a timezone and then converted to timezone-naive
for df in dfs:
	if df['DaTii'].dt.tz is None:
		df['DaTii'] = df['DaTii'].dt.tz_localize('UTC', ambiguous='NaT', nonexistent='NaT')
	df['DaTii'] = df['DaTii'].dt.tz_convert(None)

/var/folders/rt/8pf1f2_x7sj69ytpkljsggxr0000gn/T/ipykernel_54616/349464821.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[first_col] = pd.to_datetime(df[first_col], errors='coerce')
/var/folders/rt/8pf1f2_x7sj69ytpkljsggxr0000gn/T/ipykernel_54616/349464821.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[first_col] = pd.to_datetime(df[first_col], errors='coerce')


In [4]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Dictionary to store the mean and standard deviation for each dataframe
scalers = {}

# Loop through each dataframe
for i, df in enumerate(dfs):
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        print(f"Processing df{i} with columns: {numeric_cols.columns}")
        scaler = StandardScaler()
        # Fit the scaler on the dataframe and transform the data
        scaled_data = scaler.fit_transform(numeric_cols)
        
        # Save the mean and standard deviation
        scalers[f'df{i}'] = {'mean': scaler.mean_, 'scale': scaler.scale_}
        
        # Replace the original data with the standardized data
        df[numeric_cols.columns] = scaled_data
        dfs[i] = df
    else:
        print(f"df{i} has no numeric columns or is empty.")

# Assign the transformed dataframes back to their original names
df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df11 = dfs

# Save the scalers dictionary for later use
import pickle
with open('scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)

Processing df0 with columns: Index(['PM10 (Stundenmittelwerte)', 'PM2.5 (Stundenmittelwerte)'], dtype='object')
Processing df1 with columns: Index(['Rebgassechange', 'Clarahuuschange', 'Citychange', 'Storchenchange',
       'Post Baselchange', 'Aeschenchange', 'Bahnhof Südchange',
       'Bad. Bahnhofchange', 'Europechange', 'Claramattechange',
       'Elisabethenchange', 'Steinenchange', 'Kunstmuseumchange',
       'Messechange', 'Anfoschange', 'Centralbahnparkingchange'],
      dtype='object')
Processing df2 with columns: Index(['o3_stundenmittelwerte_ug_m3'], dtype='object')
Processing df3 with columns: Index(['pm10_stundenmittelwerte_ug_m3', 'pm2_5_stundenmittelwerte_ug_m3',
       'no2_stundenmittelwerte_ug_m3'],
      dtype='object')
Processing df4 with columns: Index(['O3 [ug/m3]', 'NO2 [ug/m3]', 'PM10 [ug/m3]', 'PM2.5 [ug/m3]',
       'CPC [1/cm3]', 'PREC [mm]', 'RAD [W/m2]', 'SO2 [ug/m3]',
       'NOX [ug/m3 eq. NO2]', 'TEMP [C]'],
      dtype='object')
df5 has no numeric colu

In [5]:
df11

,DaTii,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
0,2022-01-01 00:00:00,-0.612920,-0.179497,-0.470399,-0.507739
1,2022-01-01 01:00:00,-0.632843,-0.179497,-0.400364,-0.580393
2,2022-01-01 02:00:00,-0.652767,-0.179497,-0.418171,-0.696911
3,2022-01-01 03:00:00,-0.708802,-0.179497,-0.473559,-0.780239
4,2022-01-01 04:00:00,-0.747404,-0.179497,-0.470399,-0.886083
...,...,...,...,...,...
25363,2024-11-22 19:00:00,-1.632762,-0.179497,5.040528,0.731668
25364,2024-11-22 20:00:00,-1.622800,-0.179497,4.805270,0.743357
25365,2024-11-22 21:00:00,-1.629027,-0.179497,4.342020,0.769567
25366,2024-11-22 22:00:00,-1.626536,-0.179497,3.983856,0.758120


from functools import reduce

#. Merge all dataframes on 'DaTii'
merged_df = reduce(lambda left, right: pd.merge(left, right, on='DaTii', how='outer'), dfs)
merged_df